# Leaflet cluster map of talk locations

Assuming you are working in a Linux or Windows Subsystem for Linux environment, you may need to install some dependencies. Assuming a clean installation, the following will be needed:

```bash
sudo apt install jupyter
sudo apt install python3-pip
pip install python-frontmatter getorg --upgrade
```

After which you can run this from the `_talks/` directory, via:

```bash
 jupyter nbconvert --to notebook --execute talkmap.ipynb --output talkmap_out.ipynb
```
 
The `_talks/` directory contains `.md` files of all your talks. This scrapes the location YAML field from each `.md` file, geolocates it with `geopy/Nominatim`, and uses the `getorg` library to output data, HTML, and Javascript for a standalone cluster map.

In [3]:
# Start by installing the dependencies
!pip install python-frontmatter getorg --upgrade ipykernel
import frontmatter
import glob
import getorg
from geopy import Nominatim
from geopy.exc import GeocoderTimedOut

   ---------------------------------------- 138.5/138.5 kB 8.0 MB/s eta 0:00:00
   ---------------------------------------- 125.4/125.4 kB 7.2 MB/s eta 0:00:00
   --------------------------------------- 354.4/354.4 kB 21.5 MB/s eta 0:00:00
   --------------------------------------- 212.1/212.1 kB 13.5 MB/s eta 0:00:00
  Attempting uninstall: pyjwt
    Found existing installation: PyJWT 2.3.0
    Uninstalling PyJWT-2.3.0:
      Successfully uninstalled PyJWT-2.3.0
  Attempting uninstall: ipykernel
    Found existing installation: ipykernel 6.15.2
    Uninstalling ipykernel-6.15.2:
      Successfully uninstalled ipykernel-6.15.2
IPywidgets and ipyleaflet support enabled.


In [4]:
import glob

In [6]:
# Collect the Markdown files
g = glob.glob("_talks/*.md")
g

['_talks\\2022-12-AGU-1.md',
 '_talks\\2022-12-AGU-2.md',
 '_talks\\2023-03-AAG.md',
 '_talks\\2023-07-Aeolian.md',
 '_talks\\2023-12-AGU.md',
 '_talks\\2024-12-AGU-1.md',
 '_talks\\2024-12-AGU-2.md',
 '_talks\\2024-12-AGU-3.md',
 '_talks\\2024-12-AGU-4.md']

In [8]:
# Set the default timeout, in seconds
TIMEOUT = 5

# Prepare to geolocate
geocoder = Nominatim(user_agent="haijunli0629.github.io")
location_dict = {}
location = ""
permalink = ""
title = ""

In [10]:
# Read the file
file = g[0]
data = frontmatter.load(file)
data = data.to_dict()
data

{'title': 'Development of a 10 m Resolution Maize and Soybean Map Over China',
 'collection': 'talks',
 'type': 'American Geophysical Union Fall Meeting 2022',
 'permalink': '/talks/2022-12-AGU-1',
 'venue': '',
 'date': datetime.date(2022, 12, 13),
 'location': 'Chicago, Illinois, USA',
 'citation': '<b>Li, H.</b>, Song, X.P., Hansen, M.C., Becker-Reshef, I., Adusei, B., Pickering, J., Wang, L., Wang, L., Lin, Z., Zalles, V., Potapov, P., Stehman, S.V. & Justice, C.O. (2022). Development of a 10 m Resolution Maize and Soybean Map Over China. <i>AGU Fall Meeting</i>, Abstract GC23A-03, December 11-15, Chicago, Illinois, USA.',
 'content': 'Spatially explicit crop information is essential for food security and agricultural sustainability. However, high-resolution crop type maps are unavailable in most countries. In this study, we developed an experimental workflow and produced the first 10 m resolution maize and soybean map over China. To support crop mapping, we implemented a stratifie

In the event that this times out with an error, double check to make sure that the location is can be properly geolocated.

In [11]:
# Perform geolocation
for file in g:
    # Read the file
    data = frontmatter.load(file)
    data = data.to_dict()

    # Press on if the location is not present
    if 'location' not in data:
        continue

    # Prepare the description
    title = data['title'].strip()
    venue = data['venue'].strip()
    location = data['location'].strip()
    description = f"{title}<br />{venue}; {location}"

    # Geocode the location and report the status
    try:
        location_dict[description] = geocoder.geocode(location, timeout=TIMEOUT)
        print(description, location_dict[description])
    except ValueError as ex:
        print(f"Error: geocode failed on input {location} with message {ex}")
    except GeocoderTimedOut as ex:
        print(f"Error: geocode timed out on input {location} with message {ex}")
    except Exception as ex:
        print(f"An unhandled exception occurred while processing input {location} with message {ex}")

Development of a 10 m Resolution Maize and Soybean Map Over China<br />; Chicago, Illinois, USA Chicago, Cook County, Illinois, United States
Crop Type and Yield Mapping Using Long-term Satellite Observations, Weather and Field Data<br />; Chicago, Illinois, USA Chicago, Cook County, Illinois, United States
Testing global stilling with ERA5 reanalysis data<br />; Denver, Colorado, USA Denver, Colorado, United States
Decadal Changes in Relative Aeolian Transport Potential in Major Global Dust Source Regions<br />; Las Cruces, New Mexico, USA Las Cruces, Doña Ana County, New Mexico, United States
10-m Crop Mapping Using Satellite Data, Field Survey and Machine Learning over North America<br />; San Franscico, California, USA None
Advancing 10-m Crop Mapping Using All Sentinel-2 Observations over the Contiguous United States<br />; Washington, D.C., USA Washington, District of Columbia, United States
Mapping multiple crops of multiple seasons: a 30 m national crop map of Paraguay derived 

In [12]:
# Save the map
m = getorg.orgmap.create_map_obj()
getorg.orgmap.output_html_cluster_map(location_dict, folder_name="talkmap", hashed_usernames=False)

'Written map to talkmap/'